In [1]:
from transformers import pipeline, AutoModel, AutoTokenizer, BertTokenizer, BertForQuestionAnswering
import transformers
import torch
import pandas as pd
%autosave 10

Autosaving every 10 seconds


# Basic QA on covid-19 wikipedia 

In [2]:
DATA_DIRECTORY = "/home/xcs224u_student/notebooks/cs224u/github_repo/data"
nlp_qa = pipeline('question-answering')

In [3]:
nlp_qa(context='Hugging Face is a French company based in New-York.', question='Where is based Hugging Face ?')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6345.39it/s]


{'score': 0.9632967461141817, 'start': 42, 'end': 50, 'answer': 'New-York.'}

In [4]:
CONVID_ARTICLE="Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).[8] The disease was first identified in December 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.[9][10] Common symptoms include fever, cough and shortness of breath.[5] Other symptoms may include fatigue, muscle pain, diarrhoea, sore throat, loss of smell and abdominal pain.[5][11][12] The time from exposure to onset of symptoms is typically around five days, but may range from two to fourteen days.[5][13] While the majority of cases result in mild symptoms, some progress to viral pneumonia and multi-organ failure.[9][14] As of 10 April 2020, more than 1.67 million[7] cases have been reported in more than 200 countries and territories,[15] resulting in more than 101,000 deaths.[7] More than 372,000 people have recovered. The virus is mainly spread between people during close contact,[a] often via small droplets produced during coughing,[b] sneezing, or talking.[6][16][18] While these droplets are produced when breathing out, they usually fall to the ground or surfaces rather than being infectious over large distances.[6][19][20] People may also become infected by touching a contaminated surface and then their face.[6][16] The virus can survive on surfaces for up to 72 hours.[21] Coronavirus is most contagious during the first three days after onset of symptoms, although spread may be possible before symptoms appear and in later stages of the disease.The standard method of diagnosis is by real-time reverse transcription polymerase chain reaction (rRT-PCR) from a nasopharyngeal swab.[23] The infection can also be diagnosed from a combination of symptoms, risk factors and a chest CT scan showing features of pneumonia.Recommended measures to prevent infection include frequent hand washing, maintaining physical distance from others (especially from those with symptoms), covering coughs and sneezes with a tissue or inner elbow and keeping unwashed hands away from the face.[26][27] The use of masks is recommended for those who suspect they have the virus and their caregivers.[28] Recommendations for mask use by the general public vary, with some authorities recommending against their use, some recommending their use and others requiring their use.[29][30][31] Currently, there is no vaccine or specific antiviral treatment for COVID-19.[6] Management involves treatment of symptoms, supportive care, isolation and experimental measures.The World Health Organization (WHO) declared the 2019–20 coronavirus outbreak a Public Health Emergency of International Concern (PHEIC)[33][34] on 30 January 2020 and a pandemic on 11 March 2020.[10] Local transmission of the disease has been recorded in many countries across all six WHO regions.[35]"

In [5]:
nlp_qa(context=CONVID_ARTICLE, question='Where is based Hugging Face ?')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5637.51it/s]


{'score': 0.30009528747927305,
 'start': 189,
 'end': 234,
 'answer': "Wuhan, the capital of China's Hubei province,"}

In [6]:
nlp_qa(context=CONVID_ARTICLE, question='What is covid-19 ?')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5660.33it/s]


{'score': 0.25412974718281767,
 'start': 39,
 'end': 104,
 'answer': 'an infectious disease caused by severe acute respiratory syndrome'}

In [7]:
nlp_qa(context=CONVID_ARTICLE, question='What are the symptoms')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5599.87it/s]


{'score': 0.23947629162223372,
 'start': 352,
 'end': 392,
 'answer': 'fever, cough and shortness of breath.[5]'}

# get baseline model working squad

In [8]:
# we can pass file paths to from_pretrained to load pretained weights, not sure what dataset they used to train these, I think SQUAD V1
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [9]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_ids = tokenizer.encode(question, text) # add deimiter tokens, return vocab index

token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))] #

#Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]: 0 corresponds to a sentence A token,
#1 corresponds to a sentence B token

start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))

all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(input_ids)
print(token_type_ids)
print(start_scores)
print(end_scores)
print(all_tokens)
answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])

assert answer == "a nice puppet"

[101, 2040, 2001, 3958, 27227, 1029, 102, 3958, 27227, 2001, 1037, 3835, 13997, 102]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
tensor([[-4.0490, -4.1443, -5.7222, -2.4875, -5.2655, -8.2328, -4.0487,  0.3495,
         -3.8018, -1.2565,  7.1202,  5.9041,  3.4469, -4.0490]],
       grad_fn=<SqueezeBackward1>)
tensor([[ 1.1705, -4.7681, -5.0704, -4.3949, -1.5484, -4.6083,  1.1709, -3.3678,
         -0.8850, -3.4272, -0.7656,  2.3183,  7.1591,  1.1703]],
       grad_fn=<SqueezeBackward1>)
['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'nice', 'puppet', '[SEP]']


# baseline model  SQUAD using pipelines

In [10]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer


# Question answering pipeline, specifying the checkpoint identifier
qa_squad_pipeline = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='bert-base-cased')


In [11]:
qa_squad_pipeline(context='i love u, Hugging Face is a French company based in New-York.', question='Where is based Hugging Face ?')



add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5377.31it/s]


{'score': 0.9680584762547824, 'start': 52, 'end': 60, 'answer': 'New-York.'}

d

# evaluate the model on squad 2.0: Generate evaluation JSON from dev set and and feed it to evaluate-v2.0.py to obtain metrics

In [12]:
from transformers.data.processors import squad
processor = squad.SquadV2Processor()
dev_squad_2 = processor.get_dev_examples(DATA_DIRECTORY, "squad/dev-v2.0.json")
#squad datasets: https://rajpurkar.github.io/SQuAD-explorer/

100%|██████████| 35/35 [00:05<00:00,  6.19it/s]


In [13]:
import json
def save_evaluation_squad_json(squadExamples, predictions, filename=DATA_DIRECTORY+'/evaluation.json', confidence_tresh=0.5):
    obj = {};
    index =0
    for example in squadExamples:
        if predictions[index]["score"] > confidence_tresh:
            obj[example.qas_id] = predictions[index]["answer"]
        else:
            obj[example.qas_id] = ""
        index += 1;
    with open(filename, 'w') as fout:
        json.dump(obj, fout)

In [14]:
import os;
SQUAD2_BASIC_EVALUATION_FILENAME = "/squad/evaluation_dev_basic_squad2.json"
if os.path.isfile(DATA_DIRECTORY+SQUAD2_BASIC_EVALUATION_FILENAME) == False:
    print("evaluation file doesnt exist, creating it")
    qa_squad_predictions = qa_squad_pipeline(dev_squad_2);
    save_evaluation_squad_json(dev_squad_2, qa_squad_predictions, DATA_DIRECTORY+SQUAD2_BASIC_EVALUATION_FILENAME);
    

# metrics obtained for squad 2

In [15]:
# result from $ python evaluate-v2.0.py squad/dev-v2.0.json squad/evaluation_dev_basic_squad2.json

{
  "exact": 39.15606839046576,
  "f1": 43.14232380992385,
  "total": 11873,
  "HasAns_exact": 78.42442645074225,
  "HasAns_f1": 86.40836885884374,
  "HasAns_total": 5928,
  "NoAns_exact": 0.0,
  "NoAns_f1": 0.0,
  "NoAns_total": 5945
}

{'exact': 39.15606839046576,
 'f1': 43.14232380992385,
 'total': 11873,
 'HasAns_exact': 78.42442645074225,
 'HasAns_f1': 86.40836885884374,
 'HasAns_total': 5928,
 'NoAns_exact': 0.0,
 'NoAns_f1': 0.0,
 'NoAns_total': 5945}

this BERT model was finetuned on SQUAD V1 so it fails to predict no-answers, grab some examples to ilustrate this. will have to use a better model

### Squad V2 no-answer examples

In [16]:
no_answer_examples =[]
for ex in dev_squad_2:
    if(ex.is_impossible):
        no_answer_examples.append(ex)

In [17]:
len(no_answer_examples), no_answer_examples[0].start_position, no_answer_examples[0].end_position, no_answer_examples[0].qas_id





(5945, 0, 0, '5ad39d53604f3c001a3fe8d1')

In [18]:
qa_squad_pipeline(context="dasdsa", question="Who gave their name to Normandy in the 1000's and 1100's")

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5540.69it/s]


{'score': 0.5106072334642455, 'start': 0, 'end': 5, 'answer': 'dasdsa'}

{"5ad39d53604f3c001a3fe8d1": "The Normans"}
    
   

{"plausible_answers": [{"text": "Normans", "answer_start": 4}], "question": "Who gave their name to Normandy in the 1000's and 1100's", "id": "5ad39d53604f3c001a3fe8d1", "answers": [], "is_impossible": true}

# Use a model trained on squad 2 for evaluation

In [19]:
# https://huggingface.co/deepset/bert-base-cased-squad2
MODEL_NAME_SQUAD2 = "deepset/bert-base-cased-squad2"
qa_squad2_pipeline = pipeline('question-answering', model=MODEL_NAME_SQUAD2, tokenizer=MODEL_NAME_SQUAD2)


In [20]:
qa_squad_pipeline(context="Normans", question="Who gave their name to Normandy in the 1000's and 1100's")



add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5745.62it/s]


{'score': 0.571325231040003, 'start': 0, 'end': 6, 'answer': 'Normans'}

In [21]:
qa_squad2_pipeline(context="Normans", question="Who gave their name to Normandy in the 1000's and 1100's")



add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2514.57it/s]


{'score': 0.06990417145564187, 'start': 0, 'end': 6, 'answer': 'Normans'}

In [22]:
import os;
FILENAME = "/squad/evaluation_dev_sq2_squad2_2.json"
if os.path.isfile(DATA_DIRECTORY+FILENAME) == False:
    print("evaluation file doesnt exist, creating it")
    predictions = qa_squad2_pipeline(dev_squad_2);
    save_evaluation_squad_json(dev_squad_2, predictions, DATA_DIRECTORY+FILENAME);
    

## Use my modified method to make no predictions when score is too low

# Baseline model using SciBERT - pretrained with SQUAD V2

In [23]:
# https://huggingface.co/deepset/bert-base-cased-squad2
MODEL_NAME = "ktrapeznikov/scibert_scivocab_uncased_squad_v2"
qa_scibert_squad_pipeline = pipeline('question-answering', model=MODEL_NAME, tokenizer=MODEL_NAME)


# Qualitaive comparisons on covid-19

In [24]:
CONVID_ARTICLE_EASY="""Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).
The disease was first identified in December 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.
Common symptoms include fever, cough and shortness of breath.[5] Other symptoms may include fatigue, muscle pain, diarrhoea, sore throat, loss of smell and abdominal pain.[5][11][12] The time from exposure to onset of symptoms is typically around five days, but may range from two to fourteen days.[5][13] While the majority of cases result in mild symptoms, some progress to viral pneumonia and multi-organ failure.
As of 10 April 2020, more than 1.67 million[7] cases have been reported in more than 200 countries and territories,[15] resulting in more than 101,000 deaths.[7] More than 372,000 people have recovered. The virus is mainly spread between people during close contact,[a] often via small droplets produced during coughing,[b] sneezing, or talking.
While these droplets are produced when breathing out,
they usually fall to the ground or surfaces rather than being infectious over large distances.[6][19][20] People may also become infected by touching a contaminated surface and then their face.
The virus can survive on surfaces for up to 72 hours.
Coronavirus is most contagious during the first three days after onset of symptoms, although spread may be possible before symptoms
appear and in later stages of the disease.The standard method of diagnosis is by real-time reverse transcription polymerase chain reaction (rRT-PCR) from a nasopharyngeal swab.[23] The infection can also be diagnosed from a combination of symptoms, risk factors and a chest CT scan showing features of pneumonia.Recommended measures to prevent infection include frequent hand washing, maintaining physical distance from others (especially from those with symptoms), covering coughs and sneezes with a tissue or inner elbow and keeping unwashed hands away from the face.
The use of masks is recommended for those who suspect they have the virus and their caregivers.[28] Recommendations for mask use by the general public vary, with some authorities recommending against their use, some recommending their use and others requiring their use.
Currently, there is no vaccine or specific antiviral treatment for COVID-19.
Management involves treatment of symptoms, supportive care, isolation and experimental measures.The World Health Organization (WHO) declared the 2019–20 coronavirus outbreak a Public Health Emergency of International Concern (PHEIC)[33][34] on 30 January 2020 and a pandemic on 11 March 2020.[10] Local transmission of the disease has been recorded in many countries across all six WHO regions.[35]"""
CONVID_ARTICLE_MEDIUM1="""A familial cluster of 5 patients with COVID-19
pneumonia in Anyang, China, had contact before their symptom onset with an asymptomatic family member who had
traveled from the epidemic center of Wuhan. The sequence
of events suggests that the coronavirus may have been transmitted by the asymptomatic carrier. The incubation period
for patient 1 was 19 days, which is long but within the
reported range of 0 to 24 days.4 Her first RT-PCR result was
negative; false-negative results have been observed related to
the quality of the kit, the collected sample, or performance of
the test. RT-PCR has been widely deployed in diagnostic
virology and has yielded few false-positive outcomes.5 Thus,
her second RT-PCR result was unlikely to have been a falsepositive and was used to define infection with the coronavirus that causes COVID-19.
One previous study reported an asymptomatic 10-yearold boy with COVID-19 infection, but he had abnormalities
on chest CT.6 If the findings in this report of presumed transmission by an asymptomatic carrier are replicated, the prevention of COVID-19 infection would prove challenging.
The mechanism by which asymptomatic carriers could acquire and transmit the coronavirus that causes COVID-19
requires further study"""



## BERT SQUAD2

In [25]:
qa_squad2_pipeline(context=CONVID_ARTICLE_EASY, question='What are the symptoms')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5242.88it/s]


{'score': 0.24561488799397324,
 'start': 342,
 'end': 382,
 'answer': 'fever, cough and shortness of breath.[5]'}

In [26]:
qa_squad2_pipeline(context=CONVID_ARTICLE_MEDIUM1, question='does RT-PCR work')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4466.78it/s]


{'score': 0.5074767060446774,
 'start': 585,
 'end': 639,
 'answer': 'RT-PCR has been widely deployed in diagnostic virology'}

## SCIBERT SQUAD2

In [27]:
qa_scibert_squad_pipeline(context=CONVID_ARTICLE_EASY, question='What are the symptoms')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5035.18it/s]


{'score': 0.19227549805068556,
 'start': 318,
 'end': 382,
 'answer': 'Common symptoms include fever, cough and shortness of breath.[5]'}

In [29]:
qa_scibert_squad_pipeline(context=CONVID_ARTICLE_MEDIUM1, question='does RT-PCR work')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5127.51it/s]


{'score': 0.11031799662901776,
 'start': 644,
 'end': 685,
 'answer': 'has yielded few false-positive outcomes.5'}

TODO: get relevant more contexts for COVID-19 for qualitative analysis